In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
sns.set(style="darkgrid")
import sys
sys.path.append('../../src/Vrama')
sys.path.append('../../src')

from Vrama import DataController as dc
from Vrama import Tables
from Vrama import Eye

import time
from scipy import stats

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
subjectList = ['19205']

df_math = pd.DataFrame()
df_stroop = pd.DataFrame()
df_axcpt = pd.DataFrame()
for subject in subjectList:

    subj_dpath = os.path.join(dc.viveIntDataDir,subject)
    session_fpath = os.path.join(dc.viveIntDataDir,subject,subject +'_sessions.csv')
    df_sessions = pd.read_csv(session_fpath)

    for i,r in df_sessions.iterrows():

        df = pd.read_csv(os.path.join(subj_dpath,subject+"_results_"+r.Date+".csv"))
        df = FixRTandDistraction(df)
        df = df[df['distraction']=='on']
        
        if not df.empty:

            r1 = pd.DataFrame.from_dict({"Subject":[subject],"Date":[r.Date],
            "SessionType":[r.SessionType]})
            if r.TaskType == "stroop":
                r2 = Tables.Build_score_counts_stroop(df)
                df_stroop=df_stroop.append(pd.concat([r1, r2], axis=1, sort=False))
            elif r.TaskType == "math":
                r2 = Tables.Build_score_counts_math(df)
                df_math=df_math.append(pd.concat([r1, r2], axis=1, sort=False))
            elif  r.TaskType == "axcpt":
                r2 = Tables.Build_score_counts_cpt(df)
                df_axcpt=df_axcpt.append(pd.concat([r1, r2], axis=1, sort=False))
# Set index
df_stroop = df_stroop.reset_index(drop=True)
df_math = df_math.reset_index(drop=True)
df_axcpt = df_axcpt.reset_index(drop=True)

# Save to processed table folder
proc_dpath = dc.viveProDataDir
df_stroop.to_csv(os.path.join(proc_dpath,"vive_stroop_count_distraction_on_table.csv"),index=False)
df_math.to_csv(os.path.join(proc_dpath,"vive_math_count_distraction_on_table.csv"),index=False)
df_axcpt.to_csv(os.path.join(proc_dpath,"vive_cpt_count_distraction_on_table.csv"),index=False)

In [63]:
subjectList = ['19205']
cnt = 0
df_math = pd.DataFrame()
df_stroop = pd.DataFrame()
df_axcpt = pd.DataFrame()
for subject in subjectList:

    subj_dpath = os.path.join(dc.viveIntDataDir,subject)
    session_fpath = os.path.join(dc.viveIntDataDir,subject,subject +'_sessions.csv')
    df_sessions = pd.read_csv(session_fpath)

    for i,r in df_sessions.iterrows():

        df = pd.read_csv(os.path.join(subj_dpath,subject+"_results_"+r.Date+".csv"))
        df = FixRTandDistraction(df)
        df = df[df['distraction']=='off']
        
        if not df.empty:

            r1 = pd.DataFrame.from_dict({"Subject":[subject],"Date":[r.Date],
            "SessionType":[r.SessionType]})
            if r.TaskType == "stroop":
                r2 = Tables.Build_score_counts_stroop(df)
                df_stroop=df_stroop.append(pd.concat([r1, r2], axis=1, sort=False))
            elif r.TaskType == "math":
                r2 = Tables.Build_score_counts_math(df)
                df_math=df_math.append(pd.concat([r1, r2], axis=1, sort=False))
            elif  r.TaskType == "axcpt":
                if cnt == 0:
                    df_cpt0 = pd.read_csv(os.path.join(subj_dpath,subject+"_results_"+r.Date+".csv"))
                if cnt == 1:
                    df_cpt1 =pd.read_csv(os.path.join(subj_dpath,subject+"_results_"+r.Date+".csv"))
                cnt = cnt + 1
                r2 = Tables.Build_score_counts_cpt(df)
                df_axcpt=df_axcpt.append(pd.concat([r1, r2], axis=1, sort=False))
# Set index
df_stroop = df_stroop.reset_index(drop=True)
df_math = df_math.reset_index(drop=True)
df_axcpt = df_axcpt.reset_index(drop=True)

# Save to processed table folder
proc_dpath = dc.viveProDataDir
df_stroop.to_csv(os.path.join(proc_dpath,"vive_stroop_count_distraction_off_table.csv"),index=False)
df_math.to_csv(os.path.join(proc_dpath,"vive_math_count_distraction_off_table.csv"),index=False)
df_axcpt.to_csv(os.path.join(proc_dpath,"vive_cpt_count_distraction_off_table.csv"),index=False)

In [61]:
def FixRTandDistraction(df):
    for i,r in df.iterrows():
        if r.reaction_time in ['off','on']:
            r.distraction = r.reaction_time
            r.reaction_time = np.NaN
            df.iloc[i] = r
            
    return df           